In [1]:
directory = "/home/working/my_second_drive/projects/liyc_own/Zillow/"


In [2]:
import gc
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [3]:
train = pd.read_csv(directory+'train_2016.csv')
sample = pd.read_csv(directory+'sample_submission.csv')
properties = pd.read_csv(directory+'properties_2016.csv')

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
sample.shape

(2985217, 7)

In [6]:
properties.shape

(2985217, 58)

In [16]:
for c in properties.columns:
    properties[c]=properties[c].fillna(-1)
    if properties[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(properties[c].values))
        properties[c] = lbl.transform(list(properties[c].values))

In [17]:
properties.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,-1,-1,-1,0,0,-1,-1,-1,-1,...,-1,0,-1,9,2015,9,-1.00,0,-1,-1
1,10759547,-1,-1,-1,0,0,-1,-1,-1,-1,...,-1,0,-1,27516,2015,27516,-1.00,0,-1,-1
2,10843547,-1,-1,-1,0,0,-1,-1,-1,-1,...,-1,0,650756,1413387,2015,762631,20800.37,0,-1,-1
3,10859147,-1,-1,-1,0,0,3,7,-1,-1,...,1,0,571346,1156834,2015,585488,14557.57,0,-1,-1
4,10879947,-1,-1,-1,0,0,4,-1,-1,-1,...,-1,0,193796,433491,2015,239695,5725.17,0,-1,-1


In [1]:
properties.columns

NameError: name 'properties' is not defined

In [18]:
#life of property
properties['N-life'] = 2018 - properties['yearbuilt']

#error in calculation of the finished living area of home
properties['N-LivingAreaError'] = properties['calculatedfinishedsquarefeet']/properties['finishedsquarefeet12']

#proportion of living area
properties['N-LivingAreaProp'] = properties['calculatedfinishedsquarefeet']/properties['lotsizesquarefeet']
properties['N-LivingAreaProp2'] = properties['finishedsquarefeet12']/properties['finishedsquarefeet15']

#Amout of extra space
properties['N-ExtraSpace'] = properties['lotsizesquarefeet'] - properties['calculatedfinishedsquarefeet'] 
properties['N-ExtraSpace-2'] = properties['finishedsquarefeet15'] - properties['finishedsquarefeet12'] 

#Total number of rooms
properties['N-TotalRooms'] = properties['bathroomcnt']*properties['bedroomcnt']

#Average room size
properties['N-AvRoomSize'] = properties['calculatedfinishedsquarefeet']/properties['roomcnt'] 

# Number of Extra rooms
properties['N-ExtraRooms'] = properties['roomcnt'] - properties['N-TotalRooms'] 

#Ratio of the built structure value to land area
properties['N-ValueProp'] = properties['structuretaxvaluedollarcnt']/properties['landtaxvaluedollarcnt']

#Does property have a garage, pool or hot tub and AC?
properties['N-GarPoolAC'] = ((properties['garagecarcnt']>0) & (properties['pooltypeid10']>0) & (properties['airconditioningtypeid']!=5))*1 

properties["N-location"] = properties["latitude"] + properties["longitude"]
properties["N-location-2"] = properties["latitude"]*properties["longitude"]
properties["N-location-2round"] = properties["N-location-2"].round(-4)

properties["N-latitude-round"] = properties["latitude"].round(-4)
properties["N-longitude-round"] = properties["longitude"].round(-4)

#Ratio of tax of property over parcel
properties['N-ValueRatio'] = properties['taxvaluedollarcnt']/properties['taxamount']

#TotalTaxScore
properties['N-TaxScore'] = properties['taxvaluedollarcnt']*properties['taxamount']

#polnomials of tax delinquency year
properties["N-taxdelinquencyyear-2"] = properties["taxdelinquencyyear"] ** 2
properties["N-taxdelinquencyyear-3"] = properties["taxdelinquencyyear"] ** 3

#Length of time since unpaid taxes
properties['N-life-tax'] = 2018 - properties['taxdelinquencyyear']

#Indicator whether it has AC or not
properties['N-ACInd'] = (properties['airconditioningtypeid']!=5)*1

#Indicator whether it has Heating or not 
properties['N-HeatInd'] = (properties['heatingorsystemtypeid']!=13)*1

#There's 25 different property uses - let's compress them down to 4 categories
properties['N-PropType'] = properties.propertylandusetypeid.replace({31 : "Mixed", 46 : "Other", 47 : "Mixed", 246 : "Mixed", 247 : "Mixed", 248 : "Mixed", 260 : "Home", 261 : "Home", 262 : "Home", 263 : "Home", 264 : "Home", 265 : "Home", 266 : "Home", 267 : "Home", 268 : "Home", 269 : "Not Built", 270 : "Home", 271 : "Home", 273 : "Home", 274 : "Other", 275 : "Home", 276 : "Home", 279 : "Home", 290 : "Not Built", 291 : "Not Built" })


#Average structuretaxvaluedollarcnt by city
# group = properties.groupby('regionidcity')['structuretaxvaluedollarcnt'].aggregate('mean').to_dict()
# properties['N-Avg-structuretaxvaluedollarcnt'] = properties['regionidcity'].map(group)

#Deviation away from average
# properties['N-Dev-structuretaxvaluedollarcnt'] = abs((properties['structuretaxvaluedollarcnt'] - properties['N-Avg-structuretaxvaluedollarcnt']))/properties['N-Avg-structuretaxvaluedollarcnt']



In [20]:
properties.shape

(2985217, 86)

In [21]:
import pickle
f = open("properties_add_feature1.pckl","wb")
pickle.dump(properties,f)
f.close()

In [11]:
# train['transaction_month'] = pd.DatetimeIndex(train['transactiondate']).month
# df_train['transaction_month'] = df_train['transactiondate'].dt.month
# df_train['transaction_year'] = df_train['transactiondate'].dt.year



In [22]:
ofert1 = properties.groupby(['yearbuilt', 'bedroomcnt', 'regionidcity'],  as_index=False)['parcelid'].count()


In [24]:
ofert1.columns.values[3] = 'count_ParcelId'  

In [25]:
ofert1

,yearbuilt,bedroomcnt,regionidcity,count_ParcelId
0,-1,-1,-1,11442
1,-1,0,-1,3090
2,-1,0,3491,27
3,-1,0,4406,36
4,-1,0,5465,45
5,-1,0,5534,132
6,-1,0,6021,21
7,-1,0,6285,2
8,-1,0,6395,49
9,-1,0,6822,7


In [ ]:
####Ofert1: LIGTH to 0.0652813
ofert1 = prop.groupby(['yearbuilt', 'bedroomcnt', 'regionidcity'],  as_index=False)['parcelid'].count()
ofert1=pd.DataFrame(ofert1)
ofert1.columns.values[3] = 'count_ParcelId'  
prop= pd.merge(prop,ofert1, on=['yearbuilt', 'bedroomcnt', 'regionidcity'], how='left')



####Ofert2: v12
ofert2 = prop.groupby(['yearbuilt', 'roomcnt', 'regionidcity'],  as_index=False)['parcelid'].count()
ofert2=pd.DataFrame(ofert2)
ofert2.columns.values[3] = 'count_ParcelId_Of2'  
prop= pd.merge(prop,ofert2, on=['yearbuilt', 'roomcnt', 'regionidcity'], how='left')


####Ofert3:v12
ofert3 = prop.groupby(['yearbuilt', 'bathroomcnt', 'regionidcity'],  as_index=False)['parcelid'].count()
ofert3=pd.DataFrame(ofert3)
ofert3.columns.values[3] = 'count_ParcelId_Of3'  
prop= pd.merge(prop,ofert3, on=['yearbuilt', 'bathroomcnt', 'regionidcity'], how='left')


####Ofert4: v12
ofert4 = prop.groupby(['yearbuilt', 'finishedsquarefeet12', 'regionidcity'],  as_index=False)['parcelid'].count()
ofert4=pd.DataFrame(ofert4)
ofert4.columns.values[3] = 'count_ParcelId_Of4'  
prop= pd.merge(prop,ofert4, on=['yearbuilt', 'finishedsquarefeet12', 'regionidcity'], how='left')




####Tax1: ####Ofert1: LIGTH to 0.0652813

Tax1 = prop.groupby(['yearbuilt', 'bedroomcnt', 'regionidcity'],  as_index=False)['taxamount'].mean()
Tax1=pd.DataFrame(Tax1)
Tax1.columns.values[3] = 'mean_TaxAmount'  
prop= pd.merge(prop,Tax1, on=['yearbuilt', 'bedroomcnt', 'regionidcity'], how='left')